In [2]:
import pandas as pd
import numpy as np
import os
import re

## LabelBox

In [11]:
acts = '../../thais/contrato-licitações/estudo_crf/result/labelbox_atos.parquet'
entities = '../../thais/contrato-licitações/estudo_crf/result/labelbox_entidades.parquet'
texts = '../../thais/contrato-licitações/estudo_crf/result/tokenized_texts.parquet'

df1 = pd.read_parquet(acts)
df2 = pd.read_parquet(entities)
df3 = pd.read_parquet(texts)

In [12]:
tokens = df2['ner_token'].unique()

for token in tokens:
    df1[token] = np.nan

In [13]:
def clean_text_by_word(text):
    a = "\n".join([l for l in text.split("\n") if l != ""])
    words = a.replace("\n", " ").split(" ")
    words = [w for w in words if w != ""]
    
    m_words = []
    dash_cut = False

    for i in range(len(words)):
        word = words[i]

        if (word[-1] == "-") and (i+1)<len(words):
            word = word[:-1] + words[i+1]
            i += 1

        m_words.append(word)
        
    return " ".join(m_words)

In [14]:
for i in range(len(df2)):
    for j in range(len(df1)):
        if df2['arquivo_rast'][i] == df1['arquivo_rast'][j]:
            entitie = re.sub('xxbcet ?|xxbcet ?|xxeob ?|xxbob ?|xxecet ?', '', clean_text_by_word(df2['entidade'][i]).replace("\r", ""))
            df1.loc[j, df2['ner_token'][i]] = entitie

In [127]:
df1['ato'].replace({'aviso_de_licitacao': 'AVISO_LICITACAO', 
                    'extrato_de_contrato_ou_convenio': 'EXTRATO_CONTRATO_CONVENIO',
                    'aviso_de_aditamento_contratual': 'EXTRATO_ADITAMENTO_CONTRATUAL',
                    'aviso_de_suspensao_de_licitacao': 'AVISO_SUSPENSAO_LICITACAO',
                    'aviso_de_revogacao_anulacao_de_licitacao': 'AVISO_ANUL_REV_LICITACAO'}, inplace=True)

In [128]:
df1

,arquivo_rast,text,ato,dodf,treated_text,PROCESSO,NUM_AJUSTE,CONTRATANTE,CONTRATADA,OBJ_AJUSTE,...,NUM_CONTRATO,OBJ_ADITIVO,NUM_ADITIVO,IDENT_REVOGACAO_ANULACAO,IDENT_DISPENSA,SISTEMA_COMPRAS,CONVENENTE,DECISAO_TCDF,PRAZO_SUSPENSAO,OG_ATA
0,0_aviso_de_licitacao_DODF_010_15-01-2021.txt,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet PREGÃ...,AVISO_LICITACAO,DODF_010_15-01-2021,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 01/202...,00052-00021170/2020-47,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,www.comprasnet.gov.br,NaN,NaN,NaN,NaN
1,0_aviso_de_licitacao_DODF_093_20-05-2019.txt,xxbcet AVISO DE LICITAÇÃO xxecet\r\nxxbcet PRE...,AVISO_LICITACAO,DODF_093_20-05-2019,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 50/201...,0052.0001119/2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,www.comprasnet.gov.br,NaN,NaN,NaN,NaN
2,2_aviso_de_licitacao_DODF_010_15-01-2021.txt,xxbcet PREGÃO ELETRÔNICO Nº 14/2021 xxecet\nPr...,AVISO_LICITACAO,DODF_010_15-01-2021,PREGÃO ELETRÔNICO Nº 14/2021 Processo: 092.041...,092.041904/2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,www.comprasnet.gov.br,NaN,NaN,NaN,NaN
3,6_aviso_de_licitacao_DODF_002_05-01-2021.txt,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet PREGÃ...,AVISO_LICITACAO,DODF_002_05-01-2021,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO (SRP) Nº ...,00070-00004150/2020-93,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,WWwW.comprasgovernamentais.gov.br,NaN,NaN,NaN,NaN
4,9_extrato_de_contrato_ou_convenio_DODF_004_07-...,xxbcet EXTRATO DE CONTRATO SIMPLIFICADO Nº 6/2...,EXTRATO_CONTRATO_CONVENIO,DODF_004_07-01-2016,EXTRATO DE CONTRATO SIMPLIFICADO Nº 6/2015 Esp...,093.000.118/2015,6/2015,COMPANHIA ENERGÉTICA DE BRASÍLIA CEB CEB,BANCO BRADESCO S.A,Administração e controle da carteira de ações ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,100_rinst_extrato_de_aditamento_contratual_DOD...,xxbcet EXTRATO DE INSTRUMENTO CONTRATUAL xxece...,EXTRATO_ADITAMENTO_CONTRATUAL,DODF_189_02-10-2017,EXTRATO DE INSTRUMENTO CONTRATUAL PROCESSO: 11...,NaN,NaN,COMPANHIA IMOBILIÁRIA DE BRASÍLIA TERRACAP e S...,NaN,NaN,...,01/2017,"Suplementar recursos ao referido Convênio, no ...",2º Termo Aditivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1164,101_rinst_extrato_de_aditamento_contratual_DOD...,PROCESSO: 112.003.048/2016. ESPÉCIE: SEXTO TER...,EXTRATO_ADITAMENTO_CONTRATUAL,DODF_189_02-10-2017,PROCESSO: 112.003.048/2016. ESPÉCIE: SEXTO TER...,NaN,NaN,COMPANHIA URBANIZADORA DA NOVA CAPITAL DO BRAS...,NaN,NaN,...,538/2015,O objeto do presente Termo Aditivo é a prorrog...,SEXTO TERMO ADITIVO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1165,102_rinst_extrato_de_aditamento_contratual_DOD...,PROCESSO: 112.003.777/2013. ESPÉCIE: NONO TERM...,EXTRATO_ADITAMENTO_CONTRATUAL,DODF_189_02-10-2017,PROCESSO: 112.003.777/2013. ESPÉCIE: NONO TERM...,NaN,NaN,COMPANHIA URBANIZADORA DA NOVA CAPITAL DO BRAS...,NaN,NaN,...,550/2014,O objeto do presente Termo Aditivo é a prorrog...,NONO TERMO ADITIVO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1166,103_rinst_extrato_de_aditamento_contratual_DOD...,PROCESSO: 112.004.652/2015. ESPÉCIE: SEGUNDO T...,EXTRATO_ADITAMENTO_CONTRATUAL,DODF_189_02-10-2017,PROCESSO: 112.004.652/2015. ESPÉCIE: SEGUNDO T...,NaN,NaN,COMPANHIA URBANIZADORA DA NOVA CAPITAL DO BRAS...,NaN,NaN,...,559/2016,O objeto do presente Termo Aditivo é a convali...,SEGUNDO TERMO ADITIVO,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Contratos_idato

In [36]:
data = pd.read_csv('../deep-learning/acts_labeled/contratos_idato.csv')

In [37]:
data1 = data[['id_ato', 'id_dodf', 'tipo_rel', 'tipo_ent', 'texto']].loc[(data['tipo_ent'] == 'EXTRATO_ADITAMENTO_CONTRATUAL') |\
                                                               (data['tipo_ent'] == 'AVISO_LICITACAO') |\
                                                               (data['tipo_ent'] == 'AVISO_SUSPENSAO_LICITACAO') |\
                                                               (data['tipo_ent'] == 'EXTRATO_CONTRATO') |\
                                                               (data['tipo_ent'] == 'EXTRATO_CONVENIO') |\
                                                               (data['tipo_ent'] == 'AVISO_ANUL_REV_LICITACAO')]

In [38]:
data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel',
       'timestamp_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'timestamp_ent',
       'offset', 'length', 'texto']].loc[(data['tipo_ent'] != 'EXTRATO_ADITAMENTO_CONTRATUAL')]
data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel',
       'timestamp_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'timestamp_ent',
       'offset', 'length', 'texto']].loc[(data['tipo_ent'] != 'AVISO_LICITACAO')]
data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel',
       'timestamp_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'timestamp_ent',
       'offset', 'length', 'texto']].loc[(data['tipo_ent'] != 'AVISO_SUSPENSAO_LICITACAO')]
data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel',
       'timestamp_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'timestamp_ent',
       'offset', 'length', 'texto']].loc[(data['tipo_ent'] != 'EXTRATO_CONTRATO')]
data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel',
       'timestamp_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'timestamp_ent',
       'offset', 'length', 'texto']].loc[(data['tipo_ent'] != 'EXTRATO_CONVENIO')]
data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel',
       'timestamp_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'timestamp_ent',
       'offset', 'length', 'texto']].loc[(data['tipo_ent'] != 'AVISO_ANUL_REV_LICITACAO')]

In [39]:
data = data.reset_index(drop=True)
data1 = data1.reset_index(drop=True)

In [40]:
tokens = data['tipo_ent'].unique()

for token in tokens:
    data1[token] = np.nan

In [41]:
for i in range(len(data1)):
    texto = clean_text_by_word(data1.loc[i, 'texto']).replace("\r", "")
    data1.loc[i, 'texto'] = texto

In [42]:
for i in range(len(data)):
    for j in range(len(data1)):
        if data['id_ato'][i] == data1['id_ato'][j]:
            entitie = re.sub('xxbcet ?|xxbcet ?|xxeob ?|xxbob ?|xxecet ?', '', clean_text_by_word(data['texto'][i]).replace("\r", ""))
            data1.loc[j, data['tipo_ent'][i]] = entitie

In [118]:
data1['VALOR'] = np.nan

for i in range(len(data1)):
    if type(data1['valor_contrato'][i]) == str:
        data1.loc[i, 'VALOR'] = data1['valor_contrato'][i]
    elif type(data1['valor_convenio'][i]) == str:
        data1.loc[i, 'VALOR'] = data1['valor_convenio'][i]

In [119]:
data1['DATA_ASSINATURA'] = np.nan

for i in range(len(data1)):
    if type(data1['data_assinatura_contrato'][i]) == str:
        data1.loc[i, 'DATA_ASSINATURA'] = data1['data_assinatura_contrato'][i]
    elif type(data1['data_assinatura_convenio'][i]) == str:
        data1.loc[i, 'DATA_ASSINATURA'] = data1['data_assinatura_convenio'][i]

In [120]:
data1['NUM_AJUSTE'] = np.nan

for i in range(len(data1)):
    if type(data1['numero_contrato'][i]) == str:
        data1.loc[i, 'NUM_AJUSTE'] = data1['numero_contrato'][i]
    elif type(data1['numero_convenio'][i]) == str:
        data1.loc[i, 'NUM_AJUSTE'] = data1['numero_convenio'][i]

In [121]:
data1['OBJ_AJUSTE'] = np.nan

for i in range(len(data1)):
    if type(data1['objeto_contrato'][i]) == str:
        data1.loc[i, 'OBJ_AJUSTE'] = data1['objeto_contrato'][i]
    elif type(data1['objeto_convenio'][i]) == str:
        data1.loc[i, 'OBJ_AJUSTE'] = data1['objeto_convenio'][i]

In [123]:
data1['VIGENCIA'] = np.nan

for i in range(len(data1)):
    if i == 82:
        data1.loc[i, 'VIGENCIA'] = data1['vigencia_contrato'][i] + ', ' + data1['vigencia_convenio'][i]
    elif type(data1['vigencia_contrato'][i]) == str:
        data1.loc[i, 'VIGENCIA'] = data1['vigencia_contrato'][i]
    elif type(data1['vigencia_convenio'][i]) == str:
        data1.loc[i, 'VIGENCIA'] = data1['vigencia_convenio'][i]

In [44]:
data1

,id_ato,id_dodf,tipo_rel,tipo_ent,texto,numero_termo_aditivo,numero_contrato,orgao_contratante,objeto_aditamento_contratual,processo_gdf,...,entidade_convenente,cnpj_entidade_convenente,vigencia_convenio,valor_convenio,numero_convenio,objeto_convenio,data_assinatura_convenio,identificacao_ocorrencia,cnpj_orgao_contratante,cnpj_orgao_concedente
0,22_20.12.2019.1-R5,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,EXTRATO_ADITAMENTO_CONTRATUAL,EXTRATO DO QUARTO TERMO ADITIVO AO CONTRATO No...,QUARTO TERMO ADITIVO,002/2018,DF/RA II,O presente Termo Aditivo objetiva a alteracao ...,0131-000016/2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22_20.12.2019.1-R6,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,EXTRATO_ADITAMENTO_CONTRATUAL,EXTRATO DO QUARTO TERMO ADITIVO AO CONTRATO No...,QUARTO TERMO ADITIVO,002/2018,DF/RA II,O presente Termo Aditivo objetiva a alteracao ...,0131-000016/2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22_20.12.2019.1-R20,22_20.12.2019.1,REL_AVISO_LICITACAO,AVISO_LICITACAO,AVISO DE LICITACAO PREGAO ELETRONICO No 001/20...,NaN,NaN,NaN,NaN,00040-00017799/2019-42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22_20.12.2019.1-R7,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,EXTRATO_ADITAMENTO_CONTRATUAL,III TERMO ADITIVO AO CONTRATO BRB No 064/2017 ...,III TERMO ADITIVO,064/2017,BRB BANCO BANCO DE BRASILIA S/A.,acrescer qualitativamente o contrato para incl...,041.001.009/2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22_20.12.2019.1-R8,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,EXTRATO_ADITAMENTO_CONTRATUAL,Especie: Quarto Termo Aditivo ao Contrato no 1...,Quarto Termo Aditivo,114/2015,SECRETARIA DE ESTADO DE SAUDE DO DISTRITO FEDERAL,"Prorrogar o prazo de vigencia do Contrato, por...",0060-006232/2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,8_13.8.2019-R14,8_13.8.2019,REL_AVISO_LICITACAO,AVISO_LICITACAO,AVISO DE LICITACAO PREGAO ELETRONICO No 27/201...,NaN,NaN,NaN,NaN,0052.002.486/2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
722,8_13.8.2019-R6,8_13.8.2019,REL_ANUL_REVOG_LICITACAO,AVISO_ANUL_REV_LICITACAO,AVISO DE REVOGACAO DE LICITACAO PREGAO ELETRON...,NaN,NaN,NaN,NaN,00400-00023725/2019-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,revoga,NaN,NaN
723,8_13.8.2019-R13,8_13.8.2019,REL_AVISO_LICITACAO,AVISO_LICITACAO,AVISO DE LICITACAO PREGAO ELETRONICO PE No 3/2...,NaN,NaN,NaN,NaN,00400-00023725/2019-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
724,8_13.8.2019-R12,8_13.8.2019,REL_ADITAMENTO_CONTRATO,EXTRATO_ADITAMENTO_CONTRATUAL,PROCESSO: 0112-001373/2016. ESPECIE: TERCEIRO ...,TERCEIRO,566/2019,COMPANHIA URBANIZADORA DA NOVA CAPITAL DO BRAS...,"Prorrogacao do prazo de vigencia, bem como o r...",0112-001373/2016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Concatenando os dataframes

In [129]:
data1.rename({'id_dodf': 'dodf', 'tipo_ent': 'ato', 'texto': 'treated_text',
       'numero_termo_aditivo': 'NUM_ADITIVO', 'orgao_contratante': 'CONTRATANTE',
       'objeto_aditamento_contratual': 'OBJ_ADITIVO', 'processo_gdf': 'PROCESSO', 'modalidade_licitacao': 'MODALIDADE_LICITACAO',
       'numero_licitacao': 'NUM_LICITACAO', 'orgao_licitante': 'ORGAO_LICITANTE', 'sistema_compras': 'SISTEMA_COMPRAS',
       'valor_estimado_contratacao': 'VALOR_ESTIMADO', 'data_abertura_licitacao': 'DATA_ABERTURA',
       'nome_responsavel': 'NOME_RESPONSAVEL', 'tipo_objeto': 'TIPO_OBJ', 'objeto_licitacao': 'OBJ_LICITACAO', 'data_escrito': 'DATA_ESCRITO',
       'codigo_licitacao_sistema_compras': 'CODIGO_SISTEMA_COMPRAS', 'decisao_tcdf': 'DECISAO_TCDF',
       'entidade_contratada': 'CONTRATADA', 'nota_empenho': 'NE',
       'unidade_orcamentaria': 'CODIGO_UO', 'programa_trabalho': 'PT', 'natureza_despesa': 'ND',
       'fonte_recurso': 'FONTE_RECURSO',
       'cnpj_entidade_contratada': 'CNPJ_CONTRATADA', 'orgao_concedente': 'CONCEDENTE',
       'entidade_convenente': 'CONVENENTE', 'cnpj_entidade_convenente': 'CNPJ_CONVENENTE',
       'identificacao_ocorrencia': 'IDENTIFICACAO_OCORRENCIA',
       'cnpj_orgao_contratante': 'CNPJ_CONTRATANTE', 'cnpj_orgao_concedente': 'CNPJ_CONCEDENTE'}, axis=1, inplace=True)

In [134]:
data_final = data1[['ato', 'dodf', 'treated_text', 'PROCESSO',
       'NUM_AJUSTE', 'CONTRATANTE', 'CONTRATADA', 'OBJ_AJUSTE', 'VIGENCIA',
       'VALOR', 'PT', 'DATA_ASSINATURA', 'CODIGO_UO', 'ND', 'NE',
       'ORGAO_LICITANTE', 'NUM_LICITACAO', 'OBJ_LICITACAO',
       'MODALIDADE_LICITACAO', 'DATA_ABERTURA', 'TIPO_OBJ', 'VALOR_ESTIMADO',
       'CODIGO_SISTEMA_COMPRAS', 'OBJ_ADITIVO', 'NUM_ADITIVO',
       'SISTEMA_COMPRAS', 'CONVENENTE', 'DECISAO_TCDF',
       'NOME_RESPONSAVEL', 'DATA_ESCRITO', 'FONTE_RECURSO',
       'CNPJ_CONTRATADA', 'CONCEDENTE',  'CNPJ_CONVENENTE',
       'IDENTIFICACAO_OCORRENCIA', 'CNPJ_CONTRATANTE', 'CNPJ_CONCEDENTE']]

In [137]:
df_acts = pd.concat([df1, data_final])

In [139]:
df_acts = df_acts.reset_index(drop=True)

In [ ]:
for i in range(len(df_acts)):
    if type(df_acts['NUM_AJUSTE'][i]) == str:
        df_acts.loc[i, 'NUM_CONTRATO'] = df_acts['NUM_AJUSTE'][i]

In [ ]:
df_acts = df_acts[['arquivo_rast', 'text', 'ato', 'dodf', 'treated_text', 'PROCESSO',
       'CONTRATANTE', 'CONTRATADA', 'OBJ_AJUSTE', 'VIGENCIA',
       'VALOR', 'PT', 'DATA_ASSINATURA', 'CODIGO_UO', 'ND', 'NE',
       'FUND_DISPENSA', 'ORGAO_LICITANTE', 'NUM_LICITACAO', 'OBJ_LICITACAO',
       'MODALIDADE_LICITACAO', 'DATA_ABERTURA', 'TIPO_OBJ', 'VALOR_ESTIMADO',
       'CODIGO_SISTEMA_COMPRAS', 'NUM_CONTRATO', 'OBJ_ADITIVO', 'NUM_ADITIVO',
       'IDENT_REVOGACAO_ANULACAO', 'IDENT_DISPENSA', 'SISTEMA_COMPRAS',
       'CONVENENTE', 'DECISAO_TCDF', 'PRAZO_SUSPENSAO', 'OG_ATA',
       'NOME_RESPONSAVEL', 'DATA_ESCRITO', 'FONTE_RECURSO', 'CNPJ_CONTRATADA',
       'CONCEDENTE', 'CNPJ_CONVENENTE', 'IDENTIFICACAO_OCORRENCIA',
       'CNPJ_CONTRATANTE', 'CNPJ_CONCEDENTE']]

In [140]:
df_acts

,arquivo_rast,text,ato,dodf,treated_text,PROCESSO,NUM_AJUSTE,CONTRATANTE,CONTRATADA,OBJ_AJUSTE,...,OG_ATA,NOME_RESPONSAVEL,DATA_ESCRITO,FONTE_RECURSO,CNPJ_CONTRATADA,CONCEDENTE,CNPJ_CONVENENTE,IDENTIFICACAO_OCORRENCIA,CNPJ_CONTRATANTE,CNPJ_CONCEDENTE
0,0_aviso_de_licitacao_DODF_010_15-01-2021.txt,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet PREGÃ...,AVISO_LICITACAO,DODF_010_15-01-2021,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 01/202...,00052-00021170/2020-47,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0_aviso_de_licitacao_DODF_093_20-05-2019.txt,xxbcet AVISO DE LICITAÇÃO xxecet\r\nxxbcet PRE...,AVISO_LICITACAO,DODF_093_20-05-2019,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 50/201...,0052.0001119/2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2_aviso_de_licitacao_DODF_010_15-01-2021.txt,xxbcet PREGÃO ELETRÔNICO Nº 14/2021 xxecet\nPr...,AVISO_LICITACAO,DODF_010_15-01-2021,PREGÃO ELETRÔNICO Nº 14/2021 Processo: 092.041...,092.041904/2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6_aviso_de_licitacao_DODF_002_05-01-2021.txt,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet PREGÃ...,AVISO_LICITACAO,DODF_002_05-01-2021,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO (SRP) Nº ...,00070-00004150/2020-93,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9_extrato_de_contrato_ou_convenio_DODF_004_07-...,xxbcet EXTRATO DE CONTRATO SIMPLIFICADO Nº 6/2...,EXTRATO_CONTRATO_CONVENIO,DODF_004_07-01-2016,EXTRATO DE CONTRATO SIMPLIFICADO Nº 6/2015 Esp...,093.000.118/2015,6/2015,COMPANHIA ENERGÉTICA DE BRASÍLIA CEB CEB,BANCO BRADESCO S.A,Administração e controle da carteira de ações ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,NaN,NaN,AVISO_LICITACAO,8_13.8.2019,AVISO DE LICITACAO PREGAO ELETRONICO No 27/201...,0052.002.486/2017,NaN,NaN,NaN,NaN,...,NaN,KELLY CRISTINA CORDEIRO GUEDES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1890,NaN,NaN,AVISO_ANUL_REV_LICITACAO,8_13.8.2019,AVISO DE REVOGACAO DE LICITACAO PREGAO ELETRON...,00400-00023725/2019-17,NaN,NaN,NaN,NaN,...,NaN,ERONILDO DE JESUS,NaN,NaN,NaN,NaN,NaN,revoga,NaN,NaN
1891,NaN,NaN,AVISO_LICITACAO,8_13.8.2019,AVISO DE LICITACAO PREGAO ELETRONICO PE No 3/2...,00400-00023725/2019-17,NaN,NaN,NaN,NaN,...,NaN,ERONILDO DE JESUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1892,NaN,NaN,EXTRATO_ADITAMENTO_CONTRATUAL,8_13.8.2019,PROCESSO: 0112-001373/2016. ESPECIE: TERCEIRO ...,0112-001373/2016,566/2019,COMPANHIA URBANIZADORA DA NOVA CAPITAL DO BRAS...,NaN,NaN,...,NaN,NaN,09/08/2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
df_acts.isnull().sum()

arquivo_rast                 726
text                         726
ato                            0
dodf                           0
treated_text                   0
PROCESSO                     614
NUM_AJUSTE                   974
CONTRATANTE                  567
CONTRATADA                  1362
OBJ_AJUSTE                  1332
VIGENCIA                    1372
VALOR                       1436
PT                          1570
DATA_ASSINATURA             1410
CODIGO_UO                   1658
ND                          1629
NE                          1626
FUND_DISPENSA               1864
ORGAO_LICITANTE             1697
NUM_LICITACAO               1285
OBJ_LICITACAO               1450
MODALIDADE_LICITACAO        1413
DATA_ABERTURA               1496
TIPO_OBJ                    1480
VALOR_ESTIMADO              1675
CODIGO_SISTEMA_COMPRAS      1710
NUM_CONTRATO                1439
OBJ_ADITIVO                 1075
NUM_ADITIVO                 1119
IDENT_REVOGACAO_ANULACAO    1878
IDENT_DISP

In [142]:
# df_acts.to_parquet('parquet/all_acts_200x_2018_2020.parquet')